Panel is designed to give you all the flexibility you need to prototype an app or dashboard quickly and then iterate and refine it over time. Depending on your goals Panel provides several APIs that make each stage in this process as simple as possible. As a quick overview the four different APIs can be summarized as follows:

* **Interact functions**: Auto-generates a full UI (including widgets) given a function
* **Reactive functions**: Generates an reactively updating view by declaring the inputs using the ``pn.depends`` decorator
* **Parameterized class**: Generates declarative from a Parameterized class definition 
* **Callbacks**: Generate a UI by manually declaring callbacks and updating the layouts and/or views

Each of these has its own benefits and drawbacks so this section will go through each one in turn, while working through an example app and point out the benefits and drawback along the way. For a quick overview you can also review the API gallery examples, e.g. the [stocks_hvplot](../gallery/apis/stocks_hvplot.ipynb) app.

To start with let us define some imports, load the autompg dataset and define a plotting function we will be reusing throughout this user guide.

In [ ]:
import panel as pn
import hvplot.pandas
from bokeh.sampledata.autompg import autompg

def autompg_plot(x, y, color):
    return autompg.hvplot.scatter(x, y, c=color, padding=0.1)

columns = list(autompg.columns[:-2])

pn.extension()

## Interact Functions

The ``interact`` function will automatically generate a UI including widgets by inspecting the arguments of the function given to it or by providing additional hints in the ``interact`` function call. If you have worked with the [``ipywidgets``](https://github.com/jupyter-widgets/ipywidgets) package you may already be familiar with this approach (in fact the implementation is modeled on that reference implementation). The basic idea is that given a function which returns some object, Panel will inspect the arguments to that function, try to infer appropriate widgets for those arguments and then reevaluate and update the output whenever one of the widgets generate an event. For more detail on how interact generates widgets and other ways of using it see the [interact user guide](./interact.ipynb), this section instead focuses on when to use this API and its benefits and drawbacks.

The main benefit of this approach is the ease of use, you start by writing some function that returns an object, be that a plot, a dataframe or anything else that Panel can render an you immediately get an interactive UI. Unlike ipywidgets the ``pn.interact`` will also return a panel, which can be further modified by laying out the widgets and output separately. This means interact can be flexibly used to generate complex GUIs. 

#### Pros:

+ Easy to use
+ Doesn't require modifying existing code

#### Cons:

- Very implicit behavior
- Generating custom layouts requires indexing the individual components

In the example below ``pn.interact`` infers the widget type for the ``x`` argument from the type (a list) and we define explicit widgets for the ``y`` and ``color`` parameters. Next we can inspect the ``Column`` panel returned by the function (using ``.pprint()``)  and then lay it out in the way we want by indexing into the layout.

In [ ]:
y = pn.widgets.Select(value='hp', options=columns, name='y')
color = pn.widgets.ColorPicker(name='Color', value='#4f4fdf')

layout = pn.interact(autompg_plot, x=columns, y=y, color=color)

layout.pprint()

pn.Row(pn.Column('## MPG Explorer', layout[0]), layout[1])

## Reactive Functions

The reactive programming API is very similar to the ``interact`` function but makes it possible to explicitly declare the inputs to the function using the ``depends`` decorator and also makes the layout of the different components more explicit. The ``pn.depends`` decorator is a powerful way to declare the parameters a function depends on, by decorating a function with it we declare that when those parameters change the function should be called with the current values of those parameters. This makes it very explicitly which parameters the function depends on and ties that directly to some objects. Once a function has been annotated in this way it can be layed out alongside the widgets.

#### Pros:

+ Very clear mapping from the inputs to the arguments of the function
+ The layout of the different components is very explicit

#### Cons:

- Mixes the definition of the function with the GUI elements it depends on

In this model we declare all the widgets or inputs to the function first then declare the function decorated with ``pn.depends`` decorator and finally lay out the widgets and the ``autompg_plot`` function very explicitly.

In [ ]:
x = pn.widgets.Select(value='mpg', options=columns, name='x')
y = pn.widgets.Select(value='hp', options=columns, name='y')
color = pn.widgets.ColorPicker(name='Color', value='#4f4fdf')

@pn.depends(x.param.value, y.param.value, color.param.value)
def autompg_plot(x, y, color):
    return autompg.hvplot.scatter(x, y, c=color)

pn.Row(
    pn.Column('## Auto MPG Explorer', x, y, color),
    autompg_plot
)

## Parameterized Classes

The param library allows expressing the parameters of some class completely independently of a GUI implementation. Panel and other libraries can then take those parameter declarations and turn them into a GUI which controls the parameters. This allows expressing the parameters or inputs to some computation and thanks to the ``param.depends`` decorator it is then possible to directly express the dependencies between the parameters and the computation defined in some method on the class.

This is a powerful way to encapsulate some computation in a single or multiple self-contained classes taking advantage of object-oriented programming patterns. Additionally it makes it possible to express a problem completely independently from Panel or any other GUI code and then get a GUI for free as a last step. For more detail on using this approach see the [Param user guide](./Parameters.ipynb).

Pros:

+ Declarative way of declaring dependency between parameters and computation
+ The code is not tied to any particular GUI framework

Cons:

- Requires writing classes

In this model we declare a subclass of ``param.Parameterized`` declare the parameters we want at the class level, make an instance of the class  and finally lay out the parameters and plot method of the class.

In [ ]:
import param

class MPGExplorer(param.Parameterized):
    
    x = param.Selector(objects=columns)
    
    y = param.Selector(default='hp', objects=columns)
    
    color = param.Color(default='#0f0f0f')
    
    @param.depends('x', 'y', 'color')
    def plot(self):
        return autompg_plot(self.x, self.y, self.color)

explorer = MPGExplorer()

pn.Row(explorer.param, explorer.plot)

## Callbacks

The callback API in panel is the lowest level approach, which affords the greatest amount of flexibility but also quickly grows in complexity as more interactivity and therefore callbacks have to be added. Nonetheless it is an important approach to know about and can often be used to complement the other approaches, e.g. if one specific callback is needed in addition to the more reactive approaches the other APIs provide, or if each parameter should trigger very distinct changes in the output.

For more details on defining callbacks see the [Links user guide](./Links.ipynb).

#### Pros:

+ Complete and modular control over specific events

#### Cons:

- Complexity grows very quickly with the number of callbacks
- Have to handle initializing the plots separately

In this approach we once again define the widgets, unlike in other approaches we then have to define the actual layout, to ensure that the callback  we define has something that it can update or replace. In this case we use a single callback to update the plot but in many cases multiple callbacks might be required to make some change.

In [ ]:
x = pn.widgets.Select(value='mpg', options=columns, name='x')
y = pn.widgets.Select(value='hp', options=columns, name='y')
color = pn.widgets.ColorPicker(name='Color', value='#4f4fdf')

layout = pn.Row(
    pn.Column('## Auto MPG Explorer', x, y, color),
    autompg_plot(x.value, y.value, color.value)
)

def update(event):
    layout[1].object = autompg_plot(x.value, y.value, color.value)

x.param.watch(update, 'value')
y.param.watch(update, 'value')
color.param.watch(update, 'value')

layout

## Summary

As we have seen each of these four APIs allows building the same basic application. The choice of the appropriate API depends very much on the use case. To build a quick throwaway GUI the ``interact`` approach can be completely sufficient. To make that approach more explicit defining a reactive function is usually more maintainable. When writing some library code that might be used independently of the actual GUI a Parameterized class can be a great way to organize the code. Finally, if you need low-level control or want to complement any of the other approaches defining explicit callbacks can be the best approach. Choosing the API is therefore a matter of considering the tradeoffs and of course also a matter of preference.